In [1]:
import json

In [2]:
%pwd

'C:\\Users\\Bill Chen\\Desktop\\UCSD Course\\DSC180A\\DSC180A-Project1'

In [5]:
with open('config/data-params.json', 'r') as fh:
    params = json.load(fh)

In [6]:
params

{'outdir': './data/raw/',
 'train_data': ['X_1.npy', 'X_2.npy', 'X_3.npy', 'X_4.npy', 'X_5.npy'],
 'train_label': ['y_1.npy', 'y_2.npy', 'y_3.npy', 'y_4.npy', 'y_5.npy'],
 'test_data': ['X_test.npy'],
 'test_label': ['y_test.npy']}

In [7]:
import src.etl

In [8]:
train_data, train_label, test_data, test_label = src.etl.getData(**params)

In [9]:
train_data

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ...,
       [ 35,  40,  42, ...,  77,  66,  50],
       [189, 186, 185, ..., 169, 171, 171],
       [229, 236, 234, ..., 173, 162, 161]], dtype=uint8)

In [10]:
train_label

array([6, 9, 9, ..., 9, 1, 1])

In [11]:
test_data

array([[158, 159, 165, ..., 124, 129, 110],
       [235, 231, 232, ..., 178, 191, 199],
       [158, 158, 139, ...,   8,   3,   7],
       ...,
       [ 20,  19,  15, ...,  50,  53,  47],
       [ 25,  15,  23, ...,  80,  81,  80],
       [ 73,  98,  99, ...,  94,  58,  26]], dtype=uint8)

In [12]:
test_label

array([3, 8, 8, ..., 5, 1, 7])